In [1]:
import os
import yaml
import copy

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ds_static = xr.open_zarr('/glade/derecho/scratch/ksha/EPRI_data/static/static.zarr')
lon = ds_static['lon'].values
lat = ds_static['lat'].values

In [4]:
dict_loc = {
    'Pituffik': (76.4, -68.575),
    'Fairbanks': (64.75, -147.4),
    'Guam': (13.475, 144.75),
    'Yuma_PG': (33.125, -114.125),
    'Fort_Bragg': (35.05, -79.115),
}

stn_pad = 10
keys = list(dict_loc.keys())

base_dir = '/glade/derecho/scratch/ksha/EPRI_data/CESM2_SMYLE'
years = np.arange(1958, 2020)

def _wrap_lon_180(lon):
    """Map longitude to [-180, 180). Works for DataArray/ndarray/scalar."""
    return ((lon + 180) % 360) - 180

def _infer_lat_lon_names(ds):
    # Adjust candidates if your ERA5 zarr uses different naming
    lat_name = 'lat' if 'lat' in ds.coords else ('latitude' if 'latitude' in ds.coords else None)
    lon_name = 'lon' if 'lon' in ds.coords else ('longitude' if 'longitude' in ds.coords else None)
    if lat_name is None or lon_name is None:
        raise KeyError(f"Cannot find lat/lon coords. Found coords: {list(ds.coords)}")
    return lat_name, lon_name

for i, key in enumerate(keys):
    stn_lat, stn_lon = dict_loc[key]
    lon_min, lon_max = stn_lon - stn_pad, stn_lon + stn_pad
    lat_min, lat_max = stn_lat - stn_pad, stn_lat + stn_pad

    out_dir = os.path.join(base_dir, key)
    os.makedirs(out_dir, exist_ok=True)

    for year in years:
        fn = os.path.join(base_dir, f'SMYLE_{year}-11-01_daily_ensemble.zarr')
        ds = xr.open_zarr(fn)
        ds = ds.mean(['member'])
        
        lat_name, lon_name = _infer_lat_lon_names(ds)

        # ---- longitude handling (supports 0..360 or -180..180, and dateline crossing) ----
        lon = ds[lon_name]
        lon_is_360 = (lon.min() >= 0) and (lon.max() > 180)

        if lon_is_360:
            # Keep in [0, 360)
            lon_min_mod = lon_min % 360
            lon_max_mod = lon_max % 360

            if lon_min_mod <= lon_max_mod:
                lon_mask = (lon >= lon_min_mod) & (lon <= lon_max_mod)
            else:
                # crosses 0 meridian in 0..360 system
                lon_mask = (lon >= lon_min_mod) | (lon <= lon_max_mod)
        else:
            # Use [-180, 180)
            lon_wrapped = _wrap_lon_180(lon)
            lon_min_w = _wrap_lon_180(lon_min)
            lon_max_w = _wrap_lon_180(lon_max)

            if lon_min_w <= lon_max_w:
                lon_mask = (lon_wrapped >= lon_min_w) & (lon_wrapped <= lon_max_w)
            else:
                # crosses dateline in -180..180 system
                lon_mask = (lon_wrapped >= lon_min_w) | (lon_wrapped <= lon_max_w)

        lat_mask = (ds[lat_name] >= lat_min) & (ds[lat_name] <= lat_max)

        # ---- subset ----
        ds_sub = ds.sel({lat_name: lat_mask, lon_name: lon_mask})

        # Ensure coords are monotonically increasing for nicer downstream behavior
        if ds_sub[lat_name][0] > ds_sub[lat_name][-1]:
            ds_sub = ds_sub.sortby(lat_name)

        # ---- save ----
        save_name = os.path.join(out_dir, f'SMYLE_{key}_{year}.zarr')
        print(save_name)
        ds_sub.to_zarr(save_name, mode='w')

/glade/derecho/scratch/ksha/EPRI_data/CESM2_SMYLE/Pituffik/SMYLE_Pituffik_1958.zarr
/glade/derecho/scratch/ksha/EPRI_data/CESM2_SMYLE/Pituffik/SMYLE_Pituffik_1959.zarr
/glade/derecho/scratch/ksha/EPRI_data/CESM2_SMYLE/Pituffik/SMYLE_Pituffik_1960.zarr



KeyboardInterrupt



In [5]:
ds = xr.open_zarr('/glade/derecho/scratch/ksha/EPRI_data/CESM2_SMYLE/SMYLE_1958-11-01_daily_ensemble.zarr')

In [6]:
ds

<xarray.Dataset>
Dimensions:   (member: 20, time: 3711, lat: 192, lon: 288)
Coordinates:
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member    (member) int64 11 12 13 14 15 16 17 18 ... 23 24 25 26 27 28 29 30
  * time      (time) object 1958-11-01 00:00:00 ... 1968-12-31 00:00:00
Data variables:
    FLDS      (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    FSDS      (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    PRECSC    (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    PRECSL    (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    PRECT     (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    PSL       (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    QREFHT    (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    TMQ       (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    TREFHT    (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    TREFHTMN  (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    TREFHTMX  (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
    U10       (member, time, lat, lon) float32 dask.array<chunksize=(20, 6, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    case:              b.e21.BSMYLE.f09_g17.1958-11.011
    host:              cheyenne3
    initial_file:      b.e21.SMYLE_IC.f09_g17.1958-11.01.cam.i.1958-11-01-000...
    logname:           nanr
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    source:            CAM
    time_period_freq:  day_1
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...

In [7]:
ds = xr.open_zarr('/glade/derecho/scratch/ksha/EPRI_data/CESM2_grid/CESM_SYMLE_1958.zarr')

<xarray.Dataset>
Dimensions:   (time: 3711, lat: 192, lon: 288)
Coordinates:
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time      (time) object 1958-11-01 00:00:00 ... 1968-12-31 00:00:00
Data variables:
    FLDS      (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    FSDS      (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    PRECSC    (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    PRECSL    (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    PRECT     (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    PSL       (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    QREFHT    (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    TMQ       (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    TREFHT    (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    TREFHTMN  (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    TREFHTMX  (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>
    U10       (time, lat, lon) float32 dask.array<chunksize=(6, 192, 288), meta=np.ndarray>